In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
#Criando as variáveis de interesse

#Preço médio do carro Fonte: https://www.infomoney.com.br/consumo/preco-medio-do-carro-0km-chega-a-r-1514-mil-no-brasil-como-fazer-uma-boa-compra/

#Criticar post -> falta de acesso na tabela FIPE https://veiculos.fipe.org.br/

price_mean = 150000 #BRL
price_std = 10000 #BRL

price = np.random.normal(price_mean, price_std, 10000)
plt.hist(price, bins=50, density=True)

xmin, xmax = plt.xlim() # Obtém os limites do eixo x do histograma
x = np.linspace(xmin, xmax, 1000) # Cria um intervalo de valores para a curva
p = norm.pdf(x, price_mean, price_std) # Calcula a PDF da curva normal

plt.axvline(price_mean, color='k', linestyle='--', label='Média')
plt.plot(x, p, 'r', linewidth=1) # Plota a curva em vermelho com espessura 2


plt.title('Distribuição de Preços de Carros')
plt.legend(['Média','Curva Normal', 'Preços'])
plt.xlabel('Preço (R$)')
plt.ylabel('Densidade')
plt.show()
